# Process the Parallel Cycling Data

2/16/2023

In [56]:
import os, sys
from matplotlib import pyplot as plt

if os.path.basename(os.getcwd()) == 'notebooks-2023':
    os.chdir('../')
    sys.path.insert(0, 'src/')
    
import numpy as np
import pandas as pd
from scipy import interpolate
from matplotlib import pyplot as plt
%matplotlib ipympl

import src.plotter as plotter
import src.vas as vas
import src.parsers as parsers

from datetime import datetime, timedelta

%load_ext autoreload
%autoreload 2

plotter.initialize(plt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Initialize the helper class
vh = vas.VasHelper()

Initializing Voltaiq Analytic Studio Helper...
Initializing test records...
Initializing devices...
Done.


# Load the data for the pre-conditioning test for Cell A

In [146]:
vh.get_test_names('UMBL2022FEB_CELL152098')

(['UMBL2022FEB_CELL152098_CYC_1C1CR1_P45C_5P0PSI_20230217_R1-01-025',
  'UMBL2022FEB_CELL152098_RPT_1_P45C_5P0PSI_20230201_R1-01-025',
  'UMBL2022FEB_CELL152098_CYC_VMONITOR_P45C_5P0PSI_20221206_R1-01-025',
  'UMBL2022FEB_CELL152098_CYC_1C1CR1_P45C_5P0PSI_20220923_R1',
  'UMBL2022FEB_CELL152098_FORMAGING_1_P45C_5P0PSI_20220909_R1',
  'UMBL2022FEB_CELL152098_FORMTAP_2_P45C_5P0PSI_20220901_R1',
  'UMBL2022FEB_CELL152098_FORMFAST_2_P45C_5P0PSI_20220902_R1'],
 ['UMBL2022FEB_CELL152098_FORMFAST_2_P45C_P5P0PSI_20220902_R16_AuxDat_TC24_20221206_131017',
  'UMBL2022FEB_CELL152098_FORMFAST_2_P45C_P5P0PSI_20220902_R15_AuxDat_TC24_20221205_110240',
  'UMBL2022FEB_CELL152098_FORMFAST_2_P45C_P5P0PSI_20220902_R14_AuxDat_TC24_20221113_130449',
  'UMBL2022FEB_CELL152098_FORMFAST_2_P45C_P5P0PSI_20220902_R13_AuxDat_TC24_20221104_165743',
  'UMBL2022FEB_CELL152098_FORMFAST_2_P45C_P5P0PSI_20220902_R12_AuxDat_TC24_20221103_000118',
  'UMBL2022FEB_CELL152098_FORMFAST_2_P45C_P5P0PSI_20220902_R10_AuxDat_TC24_

In [147]:
cell_a_str = 'Cell 151805'

parser_cyc_a = parsers.CyclingDataParser('UMBL2022FEB_CELL151805', vh,
                                 key='UMBL2022FEB_CELL151805_CYC_1C1CR1_P45C_5P0PSI_20220804_R1',
                                 offs=0)

df_cyc_a           = parser_cyc_a.get_cycling_info()

parser_rpt_a       = parsers.RptDataParser(parser_cyc_a.df_rpt)
rpt_start_cycles_a = parser_rpt_a.get_rpt_start_cycles(offs=0)
df_rpt_a           = parser_rpt_a.get_rpt_info(rpt_start_cycles_a)

rpt_tot_discharge_capacity_ah = []
for cycle_index in df_rpt_a['cycle_index']:
    rpt_tot_discharge_capacity_ah.append(df_cyc_a['tot_discharge_capacity_ah']\
                                        [df_cyc_a.index == cycle_index].values[0])    
df_rpt_a['tot_discharge_capacity_ah'] = rpt_tot_discharge_capacity_ah

df_hppc_a = parser_rpt_a.get_hppc_info(rpt_start_cycles_a)

hppc_tot_discharge_capacity_ah = []
for cycle_index in df_hppc_a['cycle_index']:
    hppc_tot_discharge_capacity_ah.append(df_cyc_a['tot_discharge_capacity_ah']\
                                         [df_cyc_a.index == cycle_index].values[0])
    
df_hppc_a['tot_discharge_capacity_ah'] = hppc_tot_discharge_capacity_ah

Working on "UMBL2022FEB_CELL151805_CYC_1C1CR1_P45C_5P0PSI_20220804_R1"...
Initialization complete.


# Load the data for the pre-conditioning test for Cell B

In [148]:
cell_b_str = 'Cell 152098'

parser_cyc_b = parsers.CyclingDataParser('UMBL2022FEB_CELL152098', vh,
                        key='UMBL2022FEB_CELL152098_CYC_1C1CR1_P45C_5P0PSI_20220923_R1',
                        offs=0)

df_cyc_b           = parser_cyc_b.get_cycling_info()

parser_rpt_b       = parsers.RptDataParser(parser_cyc_b.df_rpt)
rpt_start_cycles_b = parser_rpt_b.get_rpt_start_cycles(offs=0)
df_rpt_b           = parser_rpt_b.get_rpt_info(rpt_start_cycles_b)

rpt_tot_discharge_capacity_ah = []
for cycle_index in df_rpt_b['cycle_index']:
    rpt_tot_discharge_capacity_ah.append(df_cyc_b['tot_discharge_capacity_ah']\
                                         [df_cyc_b.index == cycle_index].values[0])
df_rpt_b['tot_discharge_capacity_ah'] = rpt_tot_discharge_capacity_ah

df_hppc_b = parser_rpt_b.get_hppc_info(rpt_start_cycles_b)

hppc_tot_discharge_capacity_ah = []
for cycle_index in df_hppc_b['cycle_index']:
    hppc_tot_discharge_capacity_ah.append(df_cyc_b['tot_discharge_capacity_ah']\
                                         [df_cyc_b.index == cycle_index].values[0])
    
df_hppc_b['tot_discharge_capacity_ah'] = hppc_tot_discharge_capacity_ah

Working on "UMBL2022FEB_CELL152098_CYC_1C1CR1_P45C_5P0PSI_20220923_R1"...
Initialization complete.


## RPT at the end of the test

In [150]:
cell_c_str = 'Cell 151804'

parser_cyc_c = parsers.CyclingDataParser('UMBL2022FEB_CELL151804', vh,
                                 key='UMBL2022FEB_CELL151804_CYC_1C1CR1_P45C_5P0PSI_20220804_R1',
                                 offs=0)

df_cyc_c           = parser_cyc_c.get_cycling_info()

parser_rpt_c       = parsers.RptDataParser(parser_cyc_c.df_rpt)
rpt_start_cycles_c = parser_rpt_c.get_rpt_start_cycles(offs=0)
df_rpt_c           = parser_rpt_c.get_rpt_info(rpt_start_cycles_c)

rpt_tot_discharge_capacity_ah = []
for cycle_index in df_rpt_c['cycle_index']:
    rpt_tot_discharge_capacity_ah.append(df_cyc_c['tot_discharge_capacity_ah']\
                                        [df_cyc_c.index == cycle_index].values[0])    
df_rpt_c['tot_discharge_capacity_ah'] = rpt_tot_discharge_capacity_ah

df_hppc_c = parser_rpt_c.get_hppc_info(rpt_start_cycles_c)

hppc_tot_discharge_capacity_ah = []
for cycle_index in df_hppc_c['cycle_index']:
    hppc_tot_discharge_capacity_ah.append(df_cyc_c['tot_discharge_capacity_ah']\
                                         [df_cyc_c.index == cycle_index].values[0])
    
df_hppc_c['tot_discharge_capacity_ah'] = hppc_tot_discharge_capacity_ah

Working on "UMBL2022FEB_CELL151804_CYC_1C1CR1_P45C_5P0PSI_20220804_R1"...
Initialization complete.


# Load the data for the parallel test

In [151]:
cell_p_str = 'Parallel'

parser_cyc_p = parsers.CyclingDataParser('UMBL2022FEB_CELL151805', vh,
                                 key='UMBL2022FEB_CELL151805_CYC_1C1CPARALLEL_P45C_5P0PSI_20221206_R1',
                                 offs=0)

df_cyc_p           = parser_cyc_p.get_cycling_info()

parser_rpt_p       = parsers.RptDataParser(parser_cyc_p.df_rpt)
rpt_start_cycles_p = parser_rpt_p.get_rpt_start_cycles(offs=0)
df_rpt_p           = parser_rpt_p.get_rpt_info(rpt_start_cycles_p)

rpt_tot_discharge_capacity_ah = []
for cycle_index in df_rpt_p['cycle_index']:
    rpt_tot_discharge_capacity_ah.append(df_cyc_p['tot_discharge_capacity_ah']\
                                         [df_cyc_p.index == cycle_index].values[0])
df_rpt_p['tot_discharge_capacity_ah'] = rpt_tot_discharge_capacity_ah

df_hppc_p = parser_rpt_p.get_hppc_info(rpt_start_cycles_p)

hppc_tot_discharge_capacity_ah = []
for cycle_index in df_hppc_p['cycle_index']:
    hppc_tot_discharge_capacity_ah.append(df_cyc_p['tot_discharge_capacity_ah']\
                                         [df_cyc_p.index == cycle_index].values[0])
    
df_hppc_p['tot_discharge_capacity_ah'] = hppc_tot_discharge_capacity_ah

Working on "UMBL2022FEB_CELL151805_CYC_1C1CPARALLEL_P45C_5P0PSI_20221206_R1-01-005"...
Initialization complete.


## Load the RPT done on the individual cells after the parallel test hit 50% apparent capacity fade

In [175]:
parser_cyc_a2 = parsers.CyclingDataParser('UMBL2022FEB_CELL151805', vh,
                                 key='UMBL2022FEB_CELL151805_RPT_1_P45C_5P0PSI_20230201_R1-01-005',
                                 offs=0)

df_cyc_a2           = parser_cyc_a2.get_cycling_info()

parser_rpt_a2       = parsers.RptDataParser(parser_cyc_a2.df_rpt)
rpt_start_cycles_a2 = parser_rpt_a2.get_rpt_start_cycles(offs=0)
df_rpt_a2           = parser_rpt_a2.get_rpt_info(rpt_start_cycles_a2)
df_rpt_a2['tot_discharge_capacity_ah'] = df_cyc_p['tot_discharge_capacity_ah'].max()/2

df_hppc_a2 = parser_rpt_a2.get_hppc_info(rpt_start_cycles_a2)
df_hppc_a2['tot_discharge_capacity_ah'] = df_cyc_p['tot_discharge_capacity_ah'].max()/2

Working on "UMBL2022FEB_CELL151805_RPT_1_P45C_5P0PSI_20230201_R1-01-005"...
Initialization complete.


In [176]:
parser_cyc_b2 = parsers.CyclingDataParser('UMBL2022FEB_CELL152098', vh,
                                 key='UMBL2022FEB_CELL152098_RPT_1_P45C_5P0PSI_20230201_R1-01-025',
                                 offs=0)

df_cyc_b2           = parser_cyc_b2.get_cycling_info()

parser_rpt_b2       = parsers.RptDataParser(parser_cyc_b2.df_rpt)
rpt_start_cycles_b2 = parser_rpt_b2.get_rpt_start_cycles(offs=0)
df_rpt_b2           = parser_rpt_b2.get_rpt_info(rpt_start_cycles_b2)

# Offest total throughput by the parallel cycle test throughput
df_rpt_b2['tot_discharge_capacity_ah'] = df_cyc_p['tot_discharge_capacity_ah'].max()/2

df_hppc_b2 = parser_rpt_a2.get_hppc_info(rpt_start_cycles_b2)

# Offset total throughput by parallel cycle test throughput
df_hppc_b2['tot_discharge_capacity_ah'] = df_cyc_p['tot_discharge_capacity_ah'].max()/2

Working on "UMBL2022FEB_CELL152098_RPT_1_P45C_5P0PSI_20230201_R1-01-025"...
Initialization complete.


## Load the brief cycling test done on the individual cells after the parallel test hit 50% apparent capacity fade, in order to extract the 1C discharge capacity loss for each individual cell

In [172]:
parser_cyc_a3 = parsers.CyclingDataParser('UMBL2022FEB_CELL151805', vh,
                                          key='UMBL2022FEB_CELL151805_CYC_1C1CR1_P45C_5P0PSI_20230222_R1-01-005',
                                          offs=0)

df_cyc_a3 = parser_cyc_a3.get_cycling_info()

# Offset total throughput by the parallel cycle test throughput
df_cyc_a3['tot_discharge_capacity_ah'] += df_cyc_p['tot_discharge_capacity_ah'].max()/2

Working on "UMBL2022FEB_CELL151805_CYC_1C1CR1_P45C_5P0PSI_20230222_R1-01-005"...
Initialization complete.


In [173]:
parser_cyc_b3 = parsers.CyclingDataParser('UMBL2022FEB_CELL152098', vh,
                                          key='UMBL2022FEB_CELL152098_CYC_1C1CR1_P45C_5P0PSI_20230217_R1-01-025',
                                          offs=0)

# Offset total throughput by the parallel cycle test throughput
df_cyc_b3 = parser_cyc_b3.get_cycling_info()

# Offset total throughput by the parallel cycle test throughput
df_cyc_b3['tot_discharge_capacity_ah'] += df_cyc_p['tot_discharge_capacity_ah'].max()/2

Working on "UMBL2022FEB_CELL152098_CYC_1C1CR1_P45C_5P0PSI_20230217_R1-01-025"...
Initialization complete.


# Plot the data during the pre-conditioning

In [170]:
gg = (0.4, 0.4, 0.4)

plt.figure(figsize=(10, 7))
plt.plot(df_rpt_a['tot_discharge_capacity_ah']/2.5, df_rpt_a['c20_discharge_capacity_ah'], ls='--', c='r', marker='o', label=cell_a_str)
plt.plot(df_rpt_b['tot_discharge_capacity_ah']/2.5, df_rpt_b['c20_discharge_capacity_ah'], ls='--', c='b', marker='o', label=cell_b_str)
plt.plot(df_rpt_c['tot_discharge_capacity_ah']/2.5, df_rpt_c['c20_discharge_capacity_ah'], ls='--', c=gg, marker='o', label=cell_c_str)
plt.plot(df_cyc_a['tot_discharge_capacity_ah']/2.5, df_cyc_a['discharge_capacity_ah'], c='r', ls='', marker='o', ms=3, label='')
plt.plot(df_cyc_b['tot_discharge_capacity_ah']/2.5, df_cyc_b['discharge_capacity_ah'], c='b', ls='', marker='o', ms=3, label='')
plt.plot(df_cyc_c['tot_discharge_capacity_ah']/2.5, df_cyc_c['discharge_capacity_ah'], c=gg, ls='', marker='o', ms=3, label='')

plt.axhline(y=2.5*0.5, c=(0.2,0.2,0.2), label='50%')
plt.legend()
plt.ylabel('Capacity (Ah)')
plt.xlabel('EFC (Ah/Ah)')
plt.ylim((1.2, 2.6))
plt.grid(False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
plt.figure()

colors = list(reversed(plt.cm.jet(np.linspace(0,1,len(df_hppc_a['cycle_index'].unique())+1))))

for idx, cycle_index in enumerate(reversed(df_hppc_a['cycle_index'].unique())):
    
    curr_df = df_hppc_a[df_hppc_a['cycle_index'] == cycle_index]
    curr_throughput_ah = curr_df['tot_discharge_capacity_ah'].head(1).values[0]
    plt.plot(curr_df['capacity_ah'], curr_df['resistance_discharge_ohms'], label=f'{curr_throughput_ah/2.5:.0f} Ah/Ah', marker='o', color=colors[idx])
    plt.plot(curr_df['capacity_ah'], curr_df['resistance_charge_ohms'], label=f'', marker='x', linestyle='', color=colors[idx])
       
plt.legend()
plt.xlabel('Capacity (Ah)')
plt.ylabel('10s Resistance ($\Omega$)')
plt.ylim((0.02, 0.2))
plt.title(cell_a_str)

plt.figure()

colors = list(reversed(plt.cm.jet(np.linspace(0,1,len(df_hppc_b['cycle_index'].unique())+1))))

for idx, cycle_index in enumerate(reversed(df_hppc_b['cycle_index'].unique())):
    
    curr_df = df_hppc_b[df_hppc_b['cycle_index'] == cycle_index]
    curr_throughput_ah = curr_df['tot_discharge_capacity_ah'].head(1).values[0]
    plt.plot(curr_df['capacity_ah'], curr_df['resistance_discharge_ohms'], label=f'{curr_throughput_ah/2.5:.0f} Ah/Ah', marker='o', color=colors[idx])
    plt.plot(curr_df['capacity_ah'], curr_df['resistance_charge_ohms'], label=f'', marker='x', linestyle='', color=colors[idx])
       
plt.legend()
plt.xlabel('Capacity (Ah)')
plt.ylabel('10s Resistance ($\Omega$)')
plt.ylim((0.02, 0.2))
plt.title(cell_b_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Cell 152098')

In [182]:
# Plot the parallel test data
gg = (0.4, 0.4, 0.4)

plt.figure(figsize=(10, 7))
# plt.plot(df_rpt_a['tot_discharge_capacity_ah']/2.5, df_rpt_a['c20_discharge_capacity_ah'], ls='--', c='r', marker='o', label=cell_a_str)
# plt.plot(df_rpt_b['tot_discharge_capacity_ah']/2.5, df_rpt_b['c20_discharge_capacity_ah'], ls='--', c='b', marker='o', label=cell_b_str)
# plt.plot(df_rpt_c['tot_discharge_capacity_ah']/2.5, df_rpt_c['c20_discharge_capacity_ah'], ls='--', c=gg, marker='o', label=cell_c_str)
plt.plot(df_rpt_c['tot_discharge_capacity_ah']/2.5 - 428, df_rpt_c['c20_discharge_capacity_ah'], ls='--', c=gg, marker='o', label='Control')
plt.plot(df_rpt_c['tot_discharge_capacity_ah']/2.5 - 621, df_rpt_c['c20_discharge_capacity_ah'], ls='--', c=gg, marker='o', label='')
plt.plot(df_rpt_p['tot_discharge_capacity_ah']/2.5/2, df_rpt_p['c20_discharge_capacity_ah']/2, ls='--', c='g', marker='o', label=cell_p_str)

# Plot the capacity checks on individual cells after the parallel test hit 50% apparent capacity fade

# From CYC
plt.plot(df_cyc_a3['tot_discharge_capacity_ah']/2.5, df_cyc_a3['discharge_capacity_ah'], c='r', ls='', marker='o', ms=3, label='')
plt.plot(df_cyc_b3['tot_discharge_capacity_ah']/2.5, df_cyc_b3['discharge_capacity_ah'], c='b', ls='', marker='o', ms=3, label='')

# FROM RPT
plt.plot(df_rpt_a2['tot_discharge_capacity_ah']/2.5, df_rpt_a2['c20_discharge_capacity_ah'], ls='--', c='r', marker='s', markersize=10, label=cell_a_str)
plt.plot(df_rpt_b2['tot_discharge_capacity_ah']/2.5, df_rpt_b2['c20_discharge_capacity_ah'], ls='--', c='b', marker='s', markersize=10, label=cell_b_str)

# plt.plot(df_cyc_c['tot_discharge_capacity_ah']/2.5, df_cyc_c['discharge_capacity_ah'], c=gg, ls='', marker='o', ms=3, label='')


plt.plot(df_cyc_c['tot_discharge_capacity_ah']/2.5 - 428, df_cyc_c['discharge_capacity_ah'], c=gg, ls='', marker='o', ms=3, label='')
plt.plot(df_cyc_c['tot_discharge_capacity_ah']/2.5 - 621, df_cyc_c['discharge_capacity_ah'], c=gg, ls='', marker='o', ms=3, label='')
plt.plot(df_cyc_p['tot_discharge_capacity_ah']/2.5/2, df_cyc_p['discharge_capacity_ah']/2, c='g', ls='', marker='o', ms=3, label='')

plt.axhline(y=2.5*0.5, c=(0.2,0.2,0.2), label='50%')
plt.legend()
plt.ylabel('Capacity (Ah)')
plt.xlabel('EFC (Ah/Ah)')
plt.xlim((0, 700))
plt.ylim((0.7, 2.6))
plt.grid(False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Parse the current-voltage calibration curve for the hall effect current sensor

## Process the calibration curve collected on Feb 24, 2023

In [208]:
# Get the input data from the Arbin
parser_arb = parsers.CyclingDataParser('UMBL2022FEB_CELL151805', vh,
                                                 key='UMBL2022FEB_CELL151805_CYC_CURRENTRAMP_1_P45C_5P0PSI_20230224_R1-01-005',
                                                 offs=0)
df_arb_calib = parser_arb.df

Working on "UMBL2022FEB_CELL151805_CYC_CURRENTRAMP_1_P45C_5P0PSI_20230224_R1-01-005"...
Initialization complete.


In [209]:
# Get the output data from LabVIEW
df_aux_calib = vh.get_aux_data('UMBL2022FEB_CELL151805_CURRENTRAMP_1_P45C_P5P0PSI_20230224_R1_AuxDat_TC4_20230224_094846')

# Since the Arbin and the LabVIEW DAQ won't start logging at the exactly the same time, do some manual data inspection to figure out the time offset
# Also note that I haven't been successful at using Datetime to synchronize the time axis. Something weird going on beyond just a daylight savings thing (looks 
# like the data needs to be scaled, not just shifted.)
df_aux_calib['test_time_s'] += 43 

# I hooked up the hall effect sensor backwards, so flip the sign of the current.
df_aux_calib['current_a'] *= -1

In [224]:
plt.figure()
plt.plot(df_aux_calib['test_time_s'], df_aux_calib['current_a'], marker='o', label='Measured (Sensor/LabVIEW)')
plt.plot(df_arb_calib['test_time_s'], df_arb_calib['current_a'], marker='o', label='Input (Arbin)')
plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Current (A)')
plt.xlim((0,2000))
plt.setp(plt.gca().get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor');

xx = np.linspace(0, 2000, 1000)
f_y_arbin = interpolate.interp1d(df_arb_calib['test_time_s'], df_arb_calib['current_a'], kind='nearest', bounds_error=False)
f_y_labview = interpolate.interp1d(df_aux_calib['test_time_s'], df_aux_calib['current_a'], kind='nearest', bounds_error=False)

# Build the calibration curve manually
m = 0.1726
b = -0.0835

xcal = np.linspace(-2.5, 2.5, 100)
ycal = m*xcal + b

# Make the plot
plt.figure()
plt.plot(f_y_arbin(xx), f_y_labview(xx), marker='o', ls='', label='Data')
plt.plot(xcal, ycal, c='k', label=f'y={m}x+{b}')
plt.xlabel('Input Current (A)')
plt.ylabel('Output Curent (A)')
plt.legend()

# Check the result
plt.figure()
plt.plot(df_aux_calib['test_time_s'], (df_aux_calib['current_a']-b)/m, marker='o', label='Adjusted (Sensor/LabVIEW)')
plt.plot(df_arb_calib['test_time_s'], df_arb_calib['current_a'], marker='o', label='Input (Arbin)')
plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Current (A)')
plt.xlim((0,2000))
plt.setp(plt.gca().get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor');


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Parse the Current Timeseries Data

Stitch together two data sources:
1. Total current applied by Arbin, from the Arbin .res file, logged in Voltaiq
2. Branch current on Cell A, measured by the hall effect sensor and logged in Voltaiq

In [211]:
parser_main = parsers.CyclingDataParser('UMBL2022FEB_CELL151805', vh,
                                    key='UMBL2022FEB_CELL151805_CYC_1C1CPARALLEL_P45C_5P0PSI_20221206_R1-01-005',
                                    offs=0)

df_main = parser_main.df

Working on "UMBL2022FEB_CELL151805_CYC_1C1CPARALLEL_P45C_5P0PSI_20221206_R1-01-005"...
Initialization complete.


In [221]:
# Name of the Aux data file containing the current data

aux_data_name = 'UMBL2022FEB_CELL151805_CYC_1C1CPARALLEL_P45C_5P0PSI_20221206_R2_AuxDat_TC4_20221207_102709'
# aux_data_name = 'UMBL2022FEB_CELL151805_CYC_1C1CPARALLEL_P45C_5P0PSI_20221206_R1_AuxDat_TC4_20221206_133337'

df_aux = vh.get_aux_data(aux_data_name)
df_aux['test_time_s'] += 75600
df_aux['test_time_s'] += 623

# Why is b not -b here? Because the calibration data flipped the sign, and because the calibration curve is non-linear (it is affine), 
# we have to change the -b into +b to reflect the curve on the horizontal axis.
df_aux['current_a'] = (df_aux['current_a'] + b) / m 

In [226]:
# Calculate the difference, i.e. Ib = I - Ia

xx = df_aux['test_time_s']
f_y_main = interpolate.interp1d(df_main['test_time_s'], df_main['current_a'], kind='nearest', bounds_error=False)
f_y_aux = interpolate.interp1d(df_aux['test_time_s'], df_aux['current_a'], kind='nearest', bounds_error=False)

yy = f_y_main(xx) - f_y_aux(xx)

In [227]:
plt.figure()
plt.plot(df_main['test_time_s'], df_main['current_a'], label='Arbin Input', c=(0.5, 0.5, 0.5))
plt.plot(df_aux['test_time_s'], df_aux['current_a'], label='Cell 151805', c='r', marker='o', markersize=1.5, ls='')
plt.plot(xx, yy, label='Cell 152098', c='b', marker='o', markersize=1.5, ls='')
plt.xlabel('Test Time (s)')
plt.ylabel('Current (A)')
plt.grid(False)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …